# A simple quantum circuit simulation

Create a quantum circuit with 3 qubits and 3 classical bits:

In [1]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(3, 3)
# measure qubits 0, 1 & 2 to classical bits 0, 1 & 2, respectively
qc.measure([0,1,2], [0,1,2])
qc.draw()

┌─┐      
q_0: ┤M├──────
     └╥┘┌─┐   
q_1: ─╫─┤M├───
      ║ └╥┘┌─┐
q_2: ─╫──╫─┤M├
      ║  ║ └╥┘
c: 3/═╩══╩══╩═
      0  1  2

The quantum circuit includes the definition of the measurement and the storing of the measurement results in the classical bits.

Import Qiskit's simulator Aer and create new simulator object:

In [2]:
from qiskit.providers.aer import AerSimulator
sim = AerSimulator()  # make new simulator object

Run the simulation:

In [3]:
job = sim.run(qc)      # run the experiment
result = job.result()  # get the results
result.get_counts()    # interpret the results as a "counts" dictionary

{'000': 1024}

Key is a bit string, and value is the number of times this bit string was measured. Since qubits are always initialized with 0,
and we do nothing to them, we will always get '000'.

Add X gates to some of the qubits, an X gate corresponds to a classical NOT gate:

In [4]:
qc = QuantumCircuit(4, 4)
qc.x([1])  # Perform X-gates on specified qubits
qc.measure([0,1,2,3], [0,1,2,3])
qc.draw()

┌─┐      
q_0: ─────┤M├──────
     ┌───┐└╥┘   ┌─┐
q_1: ┤ X ├─╫────┤M├
     └┬─┬┘ ║    └╥┘
q_2: ─┤M├──╫─────╫─
      └╥┘  ║ ┌─┐ ║ 
q_3: ──╫───╫─┤M├─╫─
       ║   ║ └╥┘ ║ 
c: 4/══╩═══╩══╩══╩═
       2   0  3  1

In [5]:
job = sim.run(qc)
result = job.result()
result.get_counts()

{'0010': 1024}

Note that the results in the bit string are positioned from right (0) to left.

# Half and full adder

Start with the half adder first:

In [6]:
qc = QuantumCircuit(2, 2)
# Encode an input
qc.x(0)
qc.x(1)

# Apply "XOR" operation on qubits 0 and 1
qc.cx(0,1)  # CNOT controlled by qubit 0 and targeting qubit 1
qc.measure([0,1], [0,1])
display(qc.draw())

job = sim.run(qc)
result = job.result()
print("Result: ", result.get_counts())

┌───┐     ┌─┐   
q_0: ┤ X ├──■──┤M├───
     ├───┤┌─┴─┐└╥┘┌─┐
q_1: ┤ X ├┤ X ├─╫─┤M├
     └───┘└───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1

Result:  {'01': 1024}


The cx method applied to qc will overwrite qubit 1 with the result of the half-addition.

0+0 = 0, 1+0 = 0+1 = 1, 1+1 = 0

Full adder:

In [7]:
qc = QuantumCircuit(4, 2)

# Encode an input
qc.x(0)
qc.x(1)

# Carry out the adder circuit
qc.cx(0,2) # input 0 => output (qubit 2) 0, input 1 => output 1 (target flipped if control is 1)
qc.cx(1,2) # input 0 => qubit 2 remains unchanged (result of first computation), input 1 => qubit 2 flipped (again)
# At this point qubit 2 holds the result of the half-addition of qubits 0 and 1.

# Add a second gate to calculate the carryover:
qc.ccx(0,1,3)

# Measure two qubits 2 and 3 to extract the output
qc.measure(2,0)
qc.measure(3,1)
qc.draw()

┌───┐                     
q_0: ┤ X ├──■─────────■────────
     ├───┤  │         │        
q_1: ┤ X ├──┼────■────■────────
     └───┘┌─┴─┐┌─┴─┐  │  ┌─┐   
q_2: ─────┤ X ├┤ X ├──┼──┤M├───
          └───┘└───┘┌─┴─┐└╥┘┌─┐
q_3: ───────────────┤ X ├─╫─┤M├
                    └───┘ ║ └╥┘
c: 2/═════════════════════╩══╩═
                          0  1

In [8]:
job = sim.run(qc)
result = job.result()
print("Result: ", result.get_counts())

Result:  {'10': 1024}


ccx is the CCNOT or <b>Toffoli</b> gate: target qubit inverted only if both control qubits are 1.

In all examples so far the quantum circuits corresponded to classical circuits. Now we look at the real quantum stuff.

# Quantum gates and state vectors

In [14]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

qc = QuantumCircuit(2)

qc.x(0)
#qc.x(1)

# This calculates what the state vector of our qubits would be
# after passing through the circuit 'qc'
ket = Statevector(qc)

# Write down the state vector
ket.draw()

'Statevector([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],\n            dims=(2, 2))'

The state vector is the list of amplitudes of |00>, |01>, |10> and |11>. A state vector for a QC with $n$ qubits has $2^n$ components (complex amplitudes). Note that the Qiskit convention is to order qubits from right to left, i.e. |01> = (0,1,0,0) means that qubit 0 is in 1 state (|1>) and qubit 1 is in 0 state (|0>).

Now let Hadamard transformation work on qubit 1:

In [15]:
qc.h(1)

ket = Statevector(qc)
ket.draw()

'Statevector([0.        +0.j, 0.70710678+0.j, 0.        +0.j,\n             0.70710678+0.j],\n            dims=(2, 2))'

ket is now a superposition of |00> and |10>. We let the CNOT gate (cx) act on QC, with qubit 1 being the control (which is a superposition now) and qubit 0 the target qubit:

In [16]:
qc.cx(1,0)

ket = Statevector(qc)
ket.draw()

'Statevector([0.        +0.j, 0.70710678+0.j, 0.70710678+0.j,\n             0.        +0.j],\n            dims=(2, 2))'

This is an entangled state, i.e. it is not a product of two single qubits with two amplitudes: $$\frac{1}{\sqrt{2}} |00\rangle+\frac{1}{\sqrt{2}} |11\rangle$$

Disentangling can be done by applying the same operations in reverse order:

In [17]:
qc.cx(1,0)
qc.h(1)

ket = Statevector(qc)
ket.draw()

'Statevector([ 0.00000000e+00+0.j,  1.00000000e+00+0.j,  0.00000000e+00+0.j,\n             -2.23711432e-17+0.j],\n            dims=(2, 2))'